In [1]:
import numpy as np
import psana as ps
import matplotlib.pyplot as plt
import sys
import time
import pickle
import os
from Functions import *
from raw_data_class import RawData as RDC
from processed_data_class import processed_data_class as PDC
%matplotlib notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
save_dir = '/reg/d/psdm/cxi/cxilv2718/results/data/'
calib_dir = '/reg/d/psdm/cxi/cxilv2718/results/calibrations/'
runs = np.arange(86,89) # goes from first value in range to last - 1
epix_roi= [90,140,190,300]
xrt_roi = [400,1100]
calibration_file = 'calibration_2'
# Processing or Loading
process_raw = True
on_off = False # Do you want to plot the raw data?

plot_one = 87
plot_one_idx = np.searchsorted(runs,plot_one)
x_axis = 'pixels' # 'pixels' or 'energy'
to_cal_file = [calib_dir, calibration_file]

In [3]:
# Energy Calibration, plot raw calibration runs with pixel axis and populate these arrays
save_cal = False
if save_cal is True:
    calib_runs = list(range(33,41))
    notch_energies = [7.04,7.045,7.05,7.055,7.065,7.07,7.075,7.08] # keV
    xrt_pixels =  [1030,946,859,780,607,521,438,0] # enter 0 if you can't see the notch
    epix_pixels = [0,0,215,228,254,266,279,0] # enter 0 if you can't see the notch

    labels = ['notch_energies','xrt_pixels','epix_pixels','rois','to_cal_file','calib_runs']
    rois = [xrt_roi,epix_roi]
    calib_in = [labels, notch_energies,xrt_pixels,epix_pixels,rois,to_cal_file,calib_runs]
    calib_out = energy_calib.energy_calib(calib_in,save_cal)
    print('')
    print('The loaded calibration settings are: ')
    current_cal = [print(calib_out[0][i] +' : '+ str(calib_out[i+1])) for i in list(range(0,5))+list(range(8,10))]
if os.path.exists(to_cal_file[0]+to_cal_file[1]+'.pkl') and save_cal is False:
    with open(to_cal_file[0] + to_cal_file[1] +'.pkl', "rb") as f:
        calib_out = pickle.load(f)
    print('The loaded calibration settings are: ')
    current_cal = [print(calib_out[0][i] +' : '+ str(calib_out[i+1])) for i in list(range(0,5))+list(range(8,10))]   
    
if not os.path.exists(to_cal_file[0]+to_cal_file[1]+'.pkl'):
    print(to_cal_file[1] + ' does not exist.')
    print('No calibration file loaded.')

The loaded calibration settings are: 
notch_energies : [7.04, 7.045, 7.05, 7.055, 7.065, 7.07, 7.075, 7.08]
xrt_pixels : [1030, 946, 859, 780, 607, 521, 438, 0]
epix_pixels : [0, 0, 215, 228, 254, 266, 279, 0]
rois : [[400, 1100], [90, 140, 190, 300]]
to_cal_file : ['/reg/d/psdm/cxi/cxilv2718/results/calibrations/', 'calibration_2']
epix_motor : 23.237
time_calibrated : 2021-05-29 02:03:10.117061


In [ ]:
# Raw Data
plot_one = 87
raw_datas = []
for run in runs:
    scan_name = 'run_' + str(run)
    ds_string = 'exp=cxilv2718:run='+str(run)+':smd'
    print(ds_string)
    if process_raw:
        raw_data = load_data.load_data(save_dir,scan_name,ds_string,epix_roi,xrt_roi)
    else:
        if os.path.exists(save_dir + scan_name + "/" + "rawdata.pkl"):
            with open(save_dir + scan_name + "/" + "rawdata.pkl", "rb") as f:
                raw_data = pickle.load(f)
        else:
            print('You are trying to load a file that does not exist.')
    if os.path.exists(save_dir + scan_name + "/" + "rawdata.pkl"):
        raw_datas = raw_datas + [raw_data]

[load_data.add_cal_info(raw_datas[i],to_cal_file) for i in range(0,len(raw_datas))]
plot_raw.plot_raw(raw_datas, plot_one, x_axis, on_off,to_cal_file)

# plot_raw.plot_raw(raw_datas, plot_one, x_axis, on_off,to_cal_file)
# plt.figure()
# [plt.plot(raw_datas[i].epix_energy_windowed,np.mean(raw_datas[i].epix_windowed,0),label=raw_datas[i].scan_name) for i in range(0,len(runs))]
# # [plt.plot(raw_datas[i].xrt_energy_windowed,np.mean(raw_datas[i].xrt_windowed,0),label=raw_datas[i].scan_name) for i in range(0,len(runs))]

# plt.legend()
# plt.xlabel('energy,keV')
# # plt.title('XRT Spectra Stacked')
# plt.title('Epix Spectra Stacked')
# plt.show()

exp=cxilv2718:run=86:smd


In [13]:
# Applys an energy window to both spectra. Decide on window based on epix spectrum above.
# Also, reduces the resolution of the xrt spectrometer to match that of the epix.
energy_window = [7.04775,7.07725]
for i in range(0,len(raw_datas)):
    reduce_xrt_res.apply_window(raw_datas[i],energy_window)
    reduce_xrt_res.reduce_res(raw_datas[i])


# plt.figure()
# plt.plot(raw_data.epix_energy_windowed,np.sum(raw_data.xrt_red_res,0))
# plt.plot(raw_data.xrt_energy_windowed,np.sum(raw_data.xrt_windowed,0),alpha=0.5)
# plt.show()

In [14]:
# Inputs for filters
# filters = [filter0,filter1, ...,filterN]

            # filterX[0] = On/Off # True or False
            # filterX[1] = Type # 'bounds','linearity','rms'
            # filterX[2] = [par0,par1,...,parN]
            
                # If Type = 'bounds'    # par0 = 'key_0' # key_X is string from raw_data.getKeys()
                                        # par1 = lower bound # check raw_data.keyX for starting point 
                                        # par2 = upper bound # 'None' if no upper bound
                                        # par3 = number of stds from median allowed
            
                # If Type = 'linearity' # par0 = 'key_0','key_1' # key_0 and key_1 and x and y values, respectively
                                        # par1 = width of filter
                                        # par2 = force fit through origin? # True or False
                    
# filterX[3] = [in_1, in_2]
                # in_1 = On/Off # True or false
                # in_2 = plot_one # The run you want to see plots for. Declared above in the 2nd block.

# print(raw_data.getKeys()) 
suspress_output = False
filt0 = [True,'bounds',['low_diode_us',0.1,'None',3],[False,plot_one]]
filt1 = [True,'bounds',['high_diode_us',0.1,'None',2],[False,plot_one]]
filt2 = [True,'bounds',['photon_energies',0.945,'None',2],[False,plot_one]]
filt3 = [True,'bounds',['pulse_energies_fee',0.3,'None',2],[False,plot_one]]

# The linearity filters create their fit based on data points that make it through
# the bounds filters.

# filt4 = [True,'linearity',['xrt_intensity','epix_intensity',0.08,True],[True,plot_one]]
filt5 = [True,'linearity',['low_diode_us','epix_intensity',0.15,True],[False,plot_one]]
filt6 = [True,'linearity',['low_diode_us','xrt_intensity',0.15,True],[False,plot_one]]

filters = [filt0,filt1,filt2,filt3,filt5,filt6]
# filters = [filt0,filt1,filt2,filt3,filt4,filt5,filt6]

conditions = [Filters.filtering(raw_datas[i],filters,suspress_output) for i in range(0,len(raw_datas))]

Filter info for run_60:

Bounds filters removed 1 shots out of 3 total shots.
Linearity filters removed 2 shots out of 3 total shots.
The combined filters removed 3 shots out of 3 total shots | 100.0 %

Filter info for run_61:

Bounds filters removed 2 shots out of 3 total shots.
Linearity filters removed 0 shots out of 3 total shots.
The combined filters removed 2 shots out of 3 total shots | 66.667 %

Filter info for run_62:

Bounds filters removed 1028 shots out of 7859 total shots.
Linearity filters removed 924 shots out of 7859 total shots.
The combined filters removed 1952 shots out of 7859 total shots | 24.838 %

Filter info for run_63:

Bounds filters removed 1588 shots out of 12509 total shots.
Linearity filters removed 1297 shots out of 12509 total shots.
The combined filters removed 2885 shots out of 12509 total shots | 23.063 %

Filter info for run_64:

Bounds filters removed 723 shots out of 723 total shots.
Linearity filters removed 0 shots out of 723 total shots.
The com

In [15]:
# Make or load processed data
make_pro = False
pro_datas = []
probe_run = [44,make_pro]
scaling = True # True = 'avg(x)/avg(y)'| False =  'avg(x/y)' 

for i in range(0,len(raw_datas)):
    if make_pro:
        if i is np.searchsorted(runs,probe_run[0]):
            new_scale = raw_datas[np.searchsorted(runs,probe_run[0])].make_pro_data(conditions[np.searchsorted(runs,probe_run[0])],filters)
            probe_run = new_scale.scale_spectrometers(probe_run,scaling)
        processed_data = raw_datas[i].make_pro_data(conditions[i],filters)
        pro_datas = pro_datas + [processed_data]
    else:
        if os.path.exists(raw_datas[i].save_dir + raw_datas[i].scan_name + "/" + "pro_data.pkl"):
            with open(raw_datas[i].save_dir + raw_datas[i].scan_name + "/" + "pro_data.pkl", "rb") as f:
                processed_data = pickle.load(f)
                pro_datas = pro_datas + [processed_data]
                print('Loaded processed_data for run ' + processed_data.scan_name)
        else:
            print('You are trying to load a file that does not exist.')
        
    pro_datas[i].scale_spectrometers(probe_run,scaling)

len(pro_datas)

# probe_run = [26,True]
# probe_run = pro_datas[-1].scale_spectrometers(probe_run)

You are trying to load a file that does not exist.


IndexError: list index out of range

In [ ]:
# Plotting the processed data
# plot_pro      .check_SN(processed_data, On/Off): Starts off with randomizing order of shots.
                    # Each data point corresponds to the standard deviation
                    # of the mean of x residuals; epix_spectrum-xrt_spectrum.
                    # Should give us an idea of how signal/noise improves with averaging.
                    # Smoothness of curve could also report on effectiveness of filters.
# resid = plot_pro.check_SN(pro_datas[plot_one_idx],True)
# plot_pro.plot_one_run(pro_datas[plot_one_idx],True)
# plot_pro.plot_average(pro_datas,True)


# rms_parameters = [[0.8,1.2],plot_one,False] # [[min_rmse_epix, min_rmse_xrt],plot_all,plot_one]
#                                         # plot_one: Run # you want to plot, or 'None'
#                                         # plot_all: True or False to plot histograms for each run in pro_datas
# suspress_output = True

# rms_conditions = [Filters.rms_filter(pro_datas[i],rms_parameters,suspress_output) for i in range(0,len(pro_datas))]

# pro_datas = [Filters.apply_rms_filter(pro_datas[i],rms_conditions[i][2]) for i in range(0,len(pro_datas))]

In [ ]:
# total_I_epix = processed_data.epix_intensity
# total_I_xrt = processed_data.xrt_intensity
# energy = processed_data.epix_energy_windowed
# events = processed_data.eventIDs
# epix = processed_data.epix_windowed
# xrt = processed_data.xrt_based
# epix_avg = np.mean(epix,0)
# xrt_avg = np.mean(xrt,0)

# epix_norm = epix/np.max(epix_avg)
# xrt_norm = xrt/np.max(xrt_avg)
# epix_avg_norm = epix_avg/np.max(epix_avg)
# xrt_avg_norm = xrt_avg/np.max(xrt_avg)
#raw_data=RDC()

# epix_rms = np.sqrt(np.mean(np.subtract(epix_avg_norm,epix_norm)**2,1))
# xrt_rms = np.sqrt(np.mean(np.subtract(xrt_avg_norm,xrt_norm)**2,1))


# # plt.figure()
# # runs
# for i in range(0,len(runs)):
#     print(raw_datas[i].high_diode_us)
# [raw_datas[i].changeValue(epix_motor=np.round(raw_datas[i].epix_motor,3)) for i in range(0,len(raw_datas))]
##### for i in range(0,len(raw_datas)):
    
#####     with open(save_dir + raw_datas[i].scan_name + "/" + "rawdata.pkl", "wb") as f:
#####         pickle.dump(raw_datas[i], f)
# plt.figure()
# plt.plot(np.mean(processed_data.epix_norm,0))
# plt.plot(np.mean(processed_data.xrt_norm,0))
# plt.figure()
# plt.plot(np.mean(processed_data.epix_norm))
# (processed_data.epix_norm/processed_data.xrt_norm)


# rand_vals = epix[np.random.choice(epix.shape[0], np.int64(epix.shape[0]/2), replace=False)]

plot_one = 72
plot_one_idx = np.searchsorted(runs,plot_one)
epix = pro_datas[plot_one_idx].epix_norm
xrt = pro_datas[plot_one_idx].xrt_based_norm
# xrt_norm = pro_datas[plot_one_idx].xrt_norm

energy = pro_datas[plot_one_idx].epix_energy_windowed
boot = 50
plt.figure()
for i in range(0,boot):
    rand_=[np.random.choice(epix.shape[0], np.int64(epix.shape[0]/2), replace=False)]
    rand_vals_epix = epix[rand_]
    rand_vals_xrt = xrt[rand_]
    rand_residuals = np.subtract(np.mean(rand_vals_xrt,0),np.mean(rand_vals_epix,0))
    rand_DeltaT_T = np.divide(rand_residuals,np.mean(rand_vals_xrt,0))
    
#     rand_vals_xrt_norm = xrt_norm[np.random.choice(xrt.shape[0], np.int64(xrt.shape[0]/2), replace=False)]
#     rand_vals_scale = np.divide(np.mean(xrt_norm,0),np.mean(epix,0))
    
#     plt.plot(energy,rand_vals_scale)
    plt.plot(energy,np.mean(rand_vals_epix,0),'o')
    plt.plot(energy,np.mean(rand_vals_xrt,0))
#     plt.plot(energy,rand_DeltaT_T)

plt.title('Bootstrapped Epix Spectrum | run ' + str(plot_one) + ' | # Bootstraps ' + str(boot))
# plt.title('Bootstrapped XRT Spectrum | run ' + str(plot_one) + ' | # Bootstraps ' + str(boot))
# plt.title('Bootstrapped XRT-Epix  | run ' + str(plot_one) + ' | # Bootstraps ' + str(boot))

plt.xlabel('energy, eV')
plt.show()
# np.int64(len(epix)/2)
rand_residuals.shape
##################################################
# with open(processed_data.calibration_info[5][0] + 'spec_scaled_'+str(probe_run[0])+'_'+processed_data.calibration_info[5][1] + '.pkl','rb') as f:
#     spec_scale = pickle.load(f)
            
# plot_one = 42
# plot_one_idx = np.searchsorted(runs,plot_one)
# # plot_pro.plot_one_run(pro_datas[plot_one_idx],True)
# epix = pro_datas[plot_one_idx].epix_norm
# xrt = pro_datas[plot_one_idx].xrt_based_norm
# energy = pro_datas[plot_one_idx].epix_energy_windowed

# plt.figure()
# plt.plot(energy,np.mean(epix,0))
# plt.plot(energy,np.mean(xrt,0),alpha=0.5)

# plt.figure()
# plt.plot(energy,spec_scale)
print(np.mean(rand_vals_xrt,0).shape)
print(np.mean(rand_residuals,0).shape)

In [16]:
plot_one = 75
plot_one_idx = np.searchsorted(runs,plot_one)

epix = pro_datas[plot_one_idx].epix_norm
xrt = pro_datas[plot_one_idx].xrt_based_norm
runs_to_plot = [72,73,74,75]
idx = np.searchsorted(runs,runs_to_plot)
plt.figure()
[plt.plot(pro_datas[i].epix_energy_windowed, np.mean(pro_datas[i].epix_norm,0),label=pro_datas[i].scan_name)for i in idx]
plt.legend()
plt.title('Epix')

IndexError: list index out of range